# Fire Station Location Problem

In [2]:
# Standard Library Imports
import math
import base64
from typing import List, Set, Tuple, Optional, Dict, Any
import os
import time     
import itertools

# Third-Party Library Imports
import numpy as np
import pandas as pd
from PIL import Image

# Matplotlib and IPython Imports
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import display, HTML 

# Simulated Annealing

### Simulated Annealing for Minimum Dominating Set (`SimulatedAnnealingMDS`)

This class employs the Simulated Annealing (SA) metaheuristic to find an approximate solution for the Minimum Dominating Set (MDS) problem. SA navigates the solution space by iteratively transitioning between solutions, always accepting improvements and probabilistically accepting worsening moves based on a cooling "temperature," which helps avoid premature convergence to local optima.

#### Initialization Parameters:

The `SimulatedAnnealingMDS` class is configured using the following parameters:

*   **`adjacency_matrix` (np.ndarray):**
    A square matrix defining the graph structure, where an entry indicates an edge between two vertices.

*   **`initial_temp` (float):**
    The starting temperature for the annealing process. A higher value encourages broader exploration initially by increasing the likelihood of accepting worse solutions.

*   **`final_temp` (float):**
    The termination temperature. The algorithm stops when the system temperature cools down to this near-zero value.

*   **`alpha` (float):**
    The cooling rate, a factor (between 0 and 1) by which the temperature is multiplicatively reduced at each step of the cooling schedule. Values closer to 1 lead to slower cooling.

*   **`iterations` (int):**
    Defines the number of candidate solutions evaluated per "temperature stage" or acts as an overall iteration cap depending on the cooling schedule's interaction with `final_temp`.

*   **`penalty` (float, optional):**
    A value added to a solution's cost for each vertex not covered by the candidate dominating set, thereby penalizing invalid solutions.

*   **`seed` (int | None, optional):**
    An optional seed for the internal random number generator to ensure reproducible experimental results. If `None`, behavior will be non-deterministic.

In [3]:
class SimulatedAnnealingMDS:
    def __init__(
        self,
        adjacency_matrix: np.ndarray,
        initial_temp: float,
        final_temp: float,
        alpha: float,
        iterations: int,
        penalty: float = 1_000.0,
        seed: int | None = None,
    ) -> None:
        
        self.adj: np.ndarray = adjacency_matrix
        self.n_vertices: int = self.adj.shape[0]
        self.all_vertices: Set[int] = set(range(self.n_vertices))
        self.initial_temp = float(initial_temp)
        self.final_temp = float(final_temp)
        self.alpha = float(alpha)
        self.iterations = int(iterations)
        self.penalty = float(penalty)
        self.rng = np.random.default_rng(seed)
        # Closed neighborhood of v - set of all vertices directly adjacent to v plus the vertex v itself.
        self.closed_neigh: List[Set[int]] = [
            set(np.where(self.adj[v] == 1)[0]).union({v}) for v in range(self.n_vertices)
        ]
        self.temperature: float = self.initial_temp

    def initial_solution(self, strategy: str = "greedy") -> List[int]:
        if strategy not in {"greedy", "random"}:
            raise ValueError("strategy must be 'greedy' or 'random'")
        uncovered: Set[int] = set(range(self.n_vertices))
        dominating_set: Set[int] = set()

        if strategy == "greedy":
            while uncovered:
                # Find vertex that covers the most currently uncovered vertices
                # This can pick a vertex already in dominating_set if its neighborhood is large;
                # adding it again to the set is a no-op but it's chosen based on its coverage.            
                best_v_to_add = -1
                max_newly_covered_count = -1
                
                # Consider all vertices as potential candidates to add
                for v_candidate in range(self.n_vertices):
                    # If v_candidate is already in dominating_set, it won't add to |D|
                    # but its neighborhood might still cover new nodes.
                    # The key is how many currently uncovered nodes it covers.
                    count = len(self.closed_neigh[v_candidate] & uncovered)
                    if count > max_newly_covered_count:
                        max_newly_covered_count = count
                        best_v_to_add = v_candidate
                
                if best_v_to_add != -1 and max_newly_covered_count > 0 : # Found a useful vertex
                    dominating_set.add(best_v_to_add)
                    uncovered -= self.closed_neigh[best_v_to_add]
                elif uncovered: # No vertex can cover any more new uncovered nodes, but some are still uncovered
                                # This can happen in disconnected graphs or if all remaining uncovered are isolated.
                                # Add an arbitrary uncovered node to the set to make progress.
                    v_to_add = self.rng.choice(list(uncovered))
                    dominating_set.add(v_to_add)
                    uncovered -= self.closed_neigh[v_to_add]
                else: # All covered or no useful vertex found and nothing left uncovered
                    break
        else:  # random constructive heuristic
            # Pick random vertices (not necessarily from uncovered ones) until all are covered.
            nodes_to_consider = list(self.all_vertices)
            self.rng.shuffle(nodes_to_consider) # Shuffle to pick in random order

            idx = 0
            while uncovered:
                if idx >= len(nodes_to_consider): # All nodes added, but still uncovered (problematic graph)
                    break 
                v = nodes_to_consider[idx]
                idx += 1
                
                if v not in dominating_set: # Add if not already there (more efficient)
                    dominating_set.add(v)
                    uncovered -= self.closed_neigh[v]
        return list(dominating_set)

    def cost(self, candidate: List[int]) -> float:
        dom: Set[int] = set(candidate)
        if not dom: # If candidate set is empty
            return self.penalty * self.n_vertices
            
        dominated: Set[int] = set() # Initialize an empty set 'dominated' to store all vertices covered by the current candidate set 'dom'.

        for v_in_dom in dom: # Iterate through each vertex 'v_in_dom' that is part of the candidate dominating set.
            if 0 <= v_in_dom < self.n_vertices: # Check if vertex index is valid
                 dominated |= self.closed_neigh[v_in_dom] # Union the closed neighborhood of v_in_dom with dominated set.

        uncovered_count: int = self.n_vertices - len(dominated)
        return len(dom) + self.penalty * uncovered_count

    def neighbour(self, solution: List[int]) -> List[int]:
        current_sol_set: Set[int] = set(solution)
        new_sol_set = current_sol_set.copy()  # Create a copy of the current solution set to modify

        # Determine which types of moves are possible based on the current solution's size:
        can_add = len(current_sol_set) < self.n_vertices # possible if the current set doesn't already contain all vertices
        can_remove = len(current_sol_set) > 0 # possible if the current set is not empty
        can_swap = (0 < len(current_sol_set) < self.n_vertices) # possible if the set is not empty and not full

        # Create a list of possible moves based on the current solution's size:
        possible_moves = []
        if can_add: possible_moves.append("add")
        if can_remove: possible_moves.append("remove")
        if can_swap: possible_moves.append("swap")

        if not possible_moves:
            return list(new_sol_set) 

        move = self.rng.choice(possible_moves)

        if move == "add":
            non_solution_vertices = list(self.all_vertices - new_sol_set)
            if non_solution_vertices:
                v_to_add = self.rng.choice(non_solution_vertices)
                new_sol_set.add(v_to_add)
        elif move == "remove":
            solution_vertices = list(new_sol_set) # new_sol_set is not empty due to can_remove
            if solution_vertices: # Should always be true if can_remove
                v_to_remove = self.rng.choice(solution_vertices)
                new_sol_set.remove(v_to_remove)
        elif move == "swap":
            solution_vertices = list(new_sol_set)
            non_solution_vertices = list(self.all_vertices - new_sol_set)
            # Both lists must be non-empty for a swap
            if solution_vertices and non_solution_vertices:
                out_v = self.rng.choice(solution_vertices)
                in_v = self.rng.choice(non_solution_vertices)
                new_sol_set.remove(out_v)
                new_sol_set.add(in_v)
        
        return list(new_sol_set)

    def accept_probability(self, old_cost: float, new_cost: float) -> float:
        if new_cost < old_cost:
            return 1.0
        if self.temperature <= 1e-9: # Effectively zero temperature
            return 0.0
        return math.exp((old_cost - new_cost) / self.temperature)

    def search(self, init_strategy: str = "greedy", callback=None) -> Tuple[List[int], float]:
        self.temperature = self.initial_temp
        current_set: List[int] = self.initial_solution(strategy=init_strategy)
        current_cost: float = self.cost(current_set)
        best_set: List[int] = current_set.copy()
        best_cost: float = current_cost

        # Initialize the total evaluations counter to track how many candidate solutions have been evaluated.
        total_evaluations = 0

        # Main Simulated Annealing loop: continues as long as the current temperature is above the 'self.final_temp'.
        while self.temperature > self.final_temp:
            for _i_iter in range(self.iterations): # Max iterations for this "stage"

                # If temperature drops below the final threshold during these iterations, exit the inner loop early.
                if self.temperature <= self.final_temp:
                    break 

                # Generate a candidate solution by perturbing the current solution
                # and evaluate its cost.
                candidate_set = self.neighbour(current_set)
                candidate_cost = self.cost(candidate_set)

                # Accept the candidate solution with a probability based on the cost difference and current temperature.
                if self.accept_probability(current_cost, candidate_cost) > self.rng.random():
                    current_set, current_cost = candidate_set, candidate_cost

                    # If the candidate solution is better than the best found so far, update the best solution.
                    if current_cost < best_cost:
                        best_set, best_cost = current_set.copy(), current_cost
                    
                    if callback is not None:
                        # Pass relevant info to callback
                        callback(current_set, best_set, self.temperature, current_cost, best_cost)
                
                self.temperature *= self.alpha # Temperature cools after each evaluation
                total_evaluations +=1
            
            # Ensures the outer loop terminates if 'self.iterations' completes and temperature is still low.
            if self.temperature <= self.final_temp :
                break
        
        return best_set, best_cost

# Genetic Algorithm

### Genetic Algorithm for Minimum Dominating Set (`GeneticMDS`)

This class implements a Genetic Algorithm (GA) to find an approximate solution for the Minimum Dominating Set (MDS) problem. GAs are population-based metaheuristics inspired by natural selection and genetics. They evolve a population of candidate solutions over several generations using operators like selection, crossover, and mutation to search for optimal or near-optimal solutions.

#### Initialization Parameters:

The `GeneticMDS` class is configured with the following parameters:

*   **`adjacency_matrix` (np.ndarray):**
    A square matrix representing the graph's structure, where entries define connections (edges) between vertices.

*   **`pop_size` (int):**
    The number of individual candidate solutions (chromosomes) maintained in the population at each generation.

*   **`num_generations` (int):**
    The total number of generations the algorithm will run for, defining the duration of the evolutionary process.

*   **`tournament_size` (int):**
    The number of individuals randomly selected from the population to compete in a "tournament" for selection as parents. The fittest individual from the tournament is chosen.

*   **`crossover_rate` (float):**
    The probability (between 0 and 1) that two selected parent solutions will undergo crossover to produce offspring. If crossover does not occur, parents may pass to the next generation directly (after potential mutation).

*   **`mutation_rate` (float):**
    The probability (between 0 and 1) that a gene (a bit in the binary chromosome representing a vertex) will be mutated (flipped). This introduces genetic diversity. This rate is typically applied per gene.

*   **`penalty` (float, optional):**
    A value applied to the fitness (cost) function for each vertex not covered by a candidate dominating set, guiding the evolution towards valid solutions.

*   **`elitism` (bool, optional):**
    If `True`, a specified number of the best-performing individuals from the current generation are directly carried over to the next generation, ensuring their survival.

*   **`elite_size` (int, optional):**
    The number of elite individuals to preserve if `elitism` is enabled. This value is only effective if `elitism` is `True`.

*   **`use_local_search` (bool, optional):**
    If `True`, a local search heuristic is applied to individuals (typically offspring) to potentially improve them further before they join the next generation's population. This can enhance exploitation but may increase computational cost.

*   **`seed` (int | None, optional):**
    An optional seed for the internal random number generator, used to ensure reproducibility of the GA's stochastic processes. If `None`, runs will be non-deterministic.

In [4]:
import numpy as np
from typing import List, Tuple, Optional, Set # Type hinting for clarity

class GeneticMDS:
    def __init__(self,
                 adjacency_matrix: np.ndarray, 
                 pop_size: int = 100,          
                 num_generations: int = 100,   
                 tournament_size: int = 3,     
                 crossover_rate: float = 0.9,  
                 mutation_rate: float = 0.02,  
                 penalty: float = 1000.0,      
                 elitism: bool = True,        
                 elite_size: int = 2,          
                 use_local_search: bool = False,
                 seed: Optional[int] = None):

        # Validate input parameters to ensure they are within sensible ranges
        if not (0 <= crossover_rate <= 1):
            raise ValueError("crossover_rate must be in [0,1]")
        if not (0 <= mutation_rate <= 1):
            raise ValueError("mutation_rate must be in [0,1]")
        if elitism and not (1 <= elite_size < pop_size):
            raise ValueError("If elitism is True, elite_size must be in [1, pop_size)")
        if pop_size <= 0:
            raise ValueError("pop_size must be positive.")
        if num_generations < 0:
            raise ValueError("num_generations must be non-negative.")
        if not (0 < tournament_size <= pop_size):
            raise ValueError("tournament_size must be in (0, pop_size].")
        if penalty <= 0:
            raise ValueError("penalty must be positive.")

        self.adj: np.ndarray = adjacency_matrix        # Store adjacency matrix
        self.n_vertices: int = self.adj.shape[0]     # Number of vertices in the graph

        # Precompute closed neighborhoods for efficient cost calculation
        if self.n_vertices == 0: # Handle empty graph
            self.closed_neigh: List[Set[int]] = []
        else:
            # For each vertex v, its closed neighborhood is v itself and its direct neighbors
            self.closed_neigh: List[Set[int]] = [
                set(np.where(self.adj[v] == 1)[0]).union({v}) for v in range(self.n_vertices)
            ]

        # Store GA parameters
        self.pop_size = pop_size
        self.num_generations = num_generations
        self.tournament_size = tournament_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.penalty = penalty
        self.elitism = elitism
        self.elite_size = elite_size if elitism else 0 # Actual elite size based on elitism flag
        self.use_local_search = use_local_search
        self.rng = np.random.default_rng(seed) # Initialize random number generator

    def _binary_to_list(self, chromosome: np.ndarray) -> List[int]:
        """Converts a binary chromosome (0s and 1s) to a list of selected vertex indices (where bits are 1)."""
        # Finds indices where chromosome elements are 1 (using np.where on a boolean mask of 'chromosome == 1') 
        # and returns these indices as a Python list.
        return list(np.where(chromosome == 1)[0])

    def _list_to_binary(self, D_list: List[int]) -> np.ndarray:
        """Converts a list of selected vertex indices back to a binary chromosome."""
        chromosome = np.zeros(self.n_vertices, dtype=int)
        if D_list: # Avoid error if D_list is empty
            chromosome[D_list] = 1
        return chromosome

    def cost(self, chromosome: np.ndarray) -> float:
        """Calculates the cost (fitness) of a solution represented by a binary chromosome."""
        if self.n_vertices == 0: # Cost is 0 for an empty graph
            return 0.0

        selected_vertices_indices = np.where(chromosome == 1)[0] # Get indices of selected vertices
        num_selected = len(selected_vertices_indices)           # Count of selected vertices (size of dominating set)

        if num_selected == 0: # If no vertices are selected, apply a large penalty
            return self.penalty * self.n_vertices

        dominated_set: Set[int] = set() # Set to store all vertices covered by the selected ones
        for v_idx in selected_vertices_indices: # For each selected vertex
            if 0 <= v_idx < self.n_vertices: # Ensure vertex index is valid
                dominated_set.update(self.closed_neigh[v_idx]) # Add its closed neighborhood to the dominated set

        uncovered_count = self.n_vertices - len(dominated_set) # Calculate number of uncovered vertices
        # Cost = size of dominating set + penalty for each uncovered vertex
        return float(num_selected + self.penalty * uncovered_count)

    def initial_population(self) -> List[np.ndarray]:
        """Generates the initial population of random binary chromosomes."""
        population = []
        if self.n_vertices == 0: # Handle empty graph
            for _ in range(self.pop_size):
                population.append(np.array([], dtype=int))
            return population

        for _ in range(self.pop_size): # For each individual in the population
            # Create a chromosome with a random number of selected vertices (1s)
            min_select = max(1, int(0.05 * self.n_vertices)) # Select at least 1 or 5%
            max_select = min(self.n_vertices, max(min_select + 1, int(0.3 * self.n_vertices) + 1)) # Select at most 30% or N

            if min_select >= max_select: # Handle small N where min might equal max
                 num_to_select = min_select if self.n_vertices > 0 else 0
            else:
                 num_to_select = self.rng.integers(min_select, max_select) # Random number of vertices to select

            chromosome = np.zeros(self.n_vertices, dtype=int) # Initialize chromosome with all 0s
            if num_to_select > 0:
                indices = self.rng.choice(self.n_vertices, size=num_to_select, replace=False) # Randomly pick indices
                chromosome[indices] = 1 # Set selected indices to 1
            population.append(chromosome)
        return population

    def tournament_selection(self, population: List[np.ndarray], fitness_values: List[float]) -> np.ndarray:
        """Selects one parent from the population using tournament selection."""
        # Randomly choose 'tournament_size' individuals from the population
        contender_indices = self.rng.choice(len(population), size=self.tournament_size, replace=False)

        best_contender_actual_idx = -1
        min_fitness_in_tournament = float('inf') # Initialize with a very high fitness (cost)

        # Find the individual with the best (lowest) fitness among contenders
        for selected_idx in contender_indices:
            if fitness_values[selected_idx] < min_fitness_in_tournament:
                min_fitness_in_tournament = fitness_values[selected_idx]
                best_contender_actual_idx = selected_idx

        return population[best_contender_actual_idx].copy() # Return a copy of the best contender

    def one_point_crossover(self, p1: np.ndarray, p2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Performs one-point crossover between two parent chromosomes."""
        if self.n_vertices < 2: # Crossover not possible for very short chromosomes
            return p1.copy(), p2.copy()
        point = self.rng.integers(1, self.n_vertices) # Choose a random crossover point (not at the ends)
        # Create two children by swapping segments after the crossover point
        c1 = np.concatenate((p1[:point], p2[point:]))
        c2 = np.concatenate((p2[:point], p1[point:]))
        return c1, c2

    def uniform_crossover(self, p1: np.ndarray, p2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Performs uniform crossover: for each gene, randomly pick from parent 1 or parent 2."""
        c1 = p1.copy()
        c2 = p2.copy()
        for i in range(self.n_vertices): # Iterate through each gene (bit)
            if self.rng.random() < 0.5: # With 50% probability, swap the gene between children
                c1[i], c2[i] = p2[i], p1[i]
        return c1, c2

    def bit_flip_mutation(self, chromosome: np.ndarray) -> np.ndarray:
        """Performs bit-flip mutation: each bit has a 'mutation_rate' chance of being flipped."""
        mutated_chromosome = chromosome.copy()
        for i in range(self.n_vertices): # Iterate through each gene (bit)
            if self.rng.random() < self.mutation_rate: # Check if this gene should mutate
                mutated_chromosome[i] = 1 - mutated_chromosome[i] # Flip the bit (0 to 1, 1 to 0)
        return mutated_chromosome

    def local_search_mds(self, chromosome: np.ndarray) -> np.ndarray:
        """
        Applies a simple hill-climbing local search to a chromosome.
        It iteratively tries to flip single bits if the flip improves the solution's cost.
        """
        current_chromosome = chromosome.copy()
        current_cost = self.cost(current_chromosome)

        improved = True # Flag to continue search as long as improvements are found
        while improved:
            improved = False
            candidate_flips = [] # Store all possible single-bit flip neighbors
            for i in range(self.n_vertices): # For each bit in the chromosome
                flipped_chromosome = current_chromosome.copy()
                flipped_chromosome[i] = 1 - flipped_chromosome[i] # Create a neighbor by flipping the bit
                candidate_flips.append(flipped_chromosome)

            if not candidate_flips: break # Should not happen if n_vertices > 0

            costs_of_flips = [self.cost(c) for c in candidate_flips] # Calculate cost for all neighbors
            best_flip_idx = np.argmin(costs_of_flips) # Find the index of the neighbor with the best (lowest) cost

            # If the best neighbor is better than the current solution, move to it
            if costs_of_flips[best_flip_idx] < current_cost:
                current_chromosome = candidate_flips[best_flip_idx]
                current_cost = costs_of_flips[best_flip_idx]
                improved = True # Improvement found, continue local search
        return current_chromosome # Return the locally optimized chromosome

    def search(self, callback=None) -> Tuple[List[int], float]:
        """Main Genetic Algorithm search loop."""
        population = self.initial_population() # Create the initial population
        fitness_values = [self.cost(ind) for ind in population] # Calculate fitness for initial population

        if not population: # Handle empty graph
            return [], float('inf')

        # Initialize overall best solution found so far
        best_overall_chromosome: np.ndarray = population[np.argmin(fitness_values)].copy()
        best_overall_cost: float = min(fitness_values)

        # Initial callback for generation 0 (before main loop)
        if callback:
            D_curr_list = self._binary_to_list(best_overall_chromosome)
            callback(D_curr_list, D_curr_list, 0, best_overall_cost, best_overall_cost) # Pass 0 as int for gen

        # Main generational loop
        for generation in range(1, self.num_generations + 1):
            next_gen_population: List[np.ndarray] = [] # List to store individuals for the next generation

            # Elitism: Carry over the best individuals from current population
            if self.elitism:
                sorted_indices_for_elites = np.argsort(fitness_values) # Get indices sorted by fitness
                for i in range(self.elite_size):
                    next_gen_population.append(population[sorted_indices_for_elites[i]].copy())

            # Generate offspring to fill the rest of the next generation
            num_offspring_to_generate = self.pop_size - len(next_gen_population)
            offspring_generated_this_cycle: List[np.ndarray] = []

            while len(offspring_generated_this_cycle) < num_offspring_to_generate:
                # Select two parents using tournament selection
                p1 = self.tournament_selection(population, fitness_values)
                p2 = self.tournament_selection(population, fitness_values)

                # Perform crossover based on crossover_rate
                if self.rng.random() < self.crossover_rate:
                    c1, c2 = self.uniform_crossover(p1, p2) # Using uniform crossover
                    offspring_generated_this_cycle.append(c1)
                    if len(offspring_generated_this_cycle) < num_offspring_to_generate: # Add second child if space
                        offspring_generated_this_cycle.append(c2)
                else: # No crossover, parents pass through (will be mutated)
                    offspring_generated_this_cycle.append(p1.copy())
                    if len(offspring_generated_this_cycle) < num_offspring_to_generate:
                         offspring_generated_this_cycle.append(p2.copy())

            # Process each generated offspring
            for child_genome in offspring_generated_this_cycle:
                mutated_child = self.bit_flip_mutation(child_genome) # Apply mutation

                final_child = mutated_child
                if self.use_local_search: # Apply local search if enabled
                    final_child = self.local_search_mds(mutated_child)

                next_gen_population.append(final_child) # Add processed child to next generation

            population = next_gen_population # New population becomes current population
            fitness_values = [self.cost(ind) for ind in population] # Calculate fitness for new population

            # Update the overall best solution if a better one is found in this generation
            current_gen_best_idx = np.argmin(fitness_values)
            if fitness_values[current_gen_best_idx] < best_overall_cost:
                best_overall_cost = fitness_values[current_gen_best_idx]
                best_overall_chromosome = population[current_gen_best_idx].copy()

            # Call the callback function (e.g., for visualization)
            if callback is not None:
                best_of_gen_chromosome = population[current_gen_best_idx]
                D_curr_list = self._binary_to_list(best_of_gen_chromosome) # Best of current gen
                best_D_list = self._binary_to_list(best_overall_chromosome)  # Overall best
                callback(
                    D_curr_list, best_D_list, generation, # Pass generation as int
                    fitness_values[current_gen_best_idx], best_overall_cost
                )

        # Return the best solution found (as a list of indices) and its cost
        return self._binary_to_list(best_overall_chromosome), best_overall_cost

# Visualization

In [5]:
class GraphVisualizer:
    def __init__(self, adjacency_matrix, mode='local',
                pos_file='positions.csv',
                pane_aspect=(4, 3),
                pane_width_px=960,
                dpi=100):

        self.adj_matrix = np.asarray(adjacency_matrix)
        self.n = self.adj_matrix.shape[0]
        self.mode = mode
        self.ax_best = None

        panes = 2 if mode == 'local' else 1
        pane_w_in  = pane_width_px / dpi
        pane_h_in  = pane_w_in * pane_aspect[1] / pane_aspect[0]
        fig_w_in   = pane_w_in * panes
        fig_h_in   = pane_h_in

        if mode == 'local':
            self.fig, (self.ax_current, self.ax_best) = plt.subplots(
                1, 2, figsize=(fig_w_in, fig_h_in), dpi=dpi, constrained_layout=True)
        else: # Handles 'single', 'genetic', or any other mode not 'local'
            self.fig, self.ax_current = plt.subplots(
                1, 1, figsize=(fig_w_in, fig_h_in), dpi=dpi, constrained_layout=True)
            # self.ax_best remains None

        self.ax_current.set_aspect('equal')
        if self.ax_best:
            self.ax_best.set_aspect('equal')

        self.pos_file = pos_file
        self._init_positions()

    def _init_positions(self):
        loaded = False
        if self.pos_file:
            try:
                data = np.loadtxt(self.pos_file, delimiter=',')
                if data.shape[0] == self.n and data.shape[1] == 2:
                    self.pos = data
                    loaded = True
            except Exception:
                loaded = False

        if not loaded:
            positions = []
            width, height = 4.0, 3.0
            min_dist = 0.1 * max(width, height)
            if self.n > 50:
                min_dist = 0.05 * max(width, height)

            layout_rng = np.random.default_rng(seed=0) # Fixed seed for layout
            i = 0
            while i < self.n:
                x = layout_rng.random() * width
                y = layout_rng.random() * height
                if not positions or all(np.hypot(x - px, y - py) >= min_dist for px, py in positions):
                    positions.append((x, y))
                    i += 1
            self.pos = np.array(positions)

            if self.pos_file:
                try:
                    np.savetxt(self.pos_file, self.pos, delimiter=',', fmt='%.4f')
                except Exception:
                    pass # Non-critical error

    def _dominated_vertices(self, D_set_indices: List[int]) -> Set[int]: # <<< ENSURE THIS IS PRESENT TOO
        dominated = set()
        valid_D_set = {idx for idx in D_set_indices if 0 <= idx < self.n}
        for u in valid_D_set:
            dominated.add(u)
            neighbours = np.where(self.adj_matrix[u] == 1)[0]
            dominated.update(neighbours)
        return dominated

    def _draw_domset(self, ax, title: str, D_set: Set[int], dominated_set: Set[int],
                     cost: float | None, progress_value: Any | None, progress_label: str):
        ax.clear()
        full_title = title
        if cost is not None:
            full_title += f" (Cost: {cost:.2f})"

        if progress_value is not None:
            # Use the provided label and value directly
            if isinstance(progress_value, float): # <<< THIS IS THE KEY CHECK
                full_title += f" ({progress_label}: {progress_value:.2f})"
            else:
                full_title += f" ({progress_label}: {progress_value})"


        ax.set_title(full_title, fontsize=10)

        node_colors = []
        for v_node in range(self.n):
            if v_node in D_set:
                node_colors.append("#0055ff")
            elif v_node in dominated_set:
                node_colors.append('#adebad')
            else:
                node_colors.append('#ff6666')

        ax.scatter(self.pos[:,0], self.pos[:,1],
                   c=node_colors, s=80, marker='o',
                   edgecolors='gray', linewidths=0.7, zorder=3)

        for u in range(self.n):
            for v in range(u + 1, self.n):
                if self.adj_matrix[u, v] == 1:
                    is_active_edge = (u in D_set or v in D_set)
                    edge_color = 'darkgreen' if is_active_edge else 'lightgray'
                    edge_lw = 1.2 if is_active_edge else 0.6
                    edge_zorder = 2 if is_active_edge else 1
                    ax.plot([self.pos[u,0], self.pos[v,0]],
                            [self.pos[u,1], self.pos[v,1]],
                            color=edge_color, linewidth=edge_lw, zorder=edge_zorder)

        ax.set_xticks([])
        ax.set_yticks([])

    def draw_frame(self, D_curr: List[int], best_D: List[int] | None = None,
                   progress_value_curr: Any | None = None, progress_label_curr: str = "Progress",
                   progress_value_best: Any | None = None, progress_label_best: str = "Overall Best",
                   curr_cost: float | None = None, best_cost: float | None = None):
        current_D_set = set(D_curr)
        effective_best_D_set = set(best_D) if best_D is not None else current_D_set.copy()

        dom_current = self._dominated_vertices(list(current_D_set))
        dom_best    = self._dominated_vertices(list(effective_best_D_set))

        if self.mode == 'local':
            self._draw_domset(self.ax_current, f"Current Sol |D|={len(current_D_set)}",
                              current_D_set, dom_current, curr_cost,
                              progress_value_curr, progress_label_curr)
            self._draw_domset(self.ax_best,    f"Best Sol |D|={len(effective_best_D_set)}",
                              effective_best_D_set, dom_best, best_cost,
                              progress_value_best, progress_label_best)
        else:
            self._draw_domset(self.ax_current, f"Best Sol |D|={len(effective_best_D_set)}",
                              effective_best_D_set, dom_best, best_cost,
                              progress_value_best, progress_label_best)

        self.fig.canvas.draw_idle()

In [6]:
def run_and_visualize_algorithm(
    algorithm_instance: Any,
    algorithm_name: str,
    adjacency_matrix_data: np.ndarray,
    num_vertices: int,
    search_kwargs: Optional[Dict[str, Any]] = None,
    vis_pane_width_px: int = 400,
    vis_dpi: int = 70,
    vis_pos_file_template: str = "mds_positions_v{N_VERTICES}.csv",
    gif_base_filename_template: str = "run_{ALGO_NAME_SLUG}", # Base name, params will be added
    gif_output_folder: Optional[str] = None, # New: Folder to save GIFs
    mode: str = 'local',
    generate_gif: bool = True,
    current_config_params_for_gif_name: Optional[Dict[str, Any]] = None # New: To make GIF names unique
) -> Tuple[List[int], float]:
    """
    Runs an optimization algorithm, optionally visualizes, saves GIF, and returns results.
    """
    visualizer = None
    collected_frames: List[np.ndarray] = []
    actual_gif_filepath = None # To store the full path of the generated GIF

    if generate_gif:
        pos_file = vis_pos_file_template.format(N_VERTICES=num_vertices)
        visualizer = GraphVisualizer(
            adjacency_matrix_data, mode=mode, pane_width_px=vis_pane_width_px,
            dpi=vis_dpi, pos_file=pos_file
        )
        progress_label = "Iter"
        if "Simulated Annealing" in algorithm_name: progress_label = "Temp"
        elif "Genetic Algorithm" in algorithm_name: progress_label = "Gen"

        def generic_emit_frame(d_curr, d_best, p_metric, c_curr, c_best):
            if visualizer:
                # ... (same emit_frame logic as before, calling visualizer.draw_frame) ...
                if mode == 'local':
                    visualizer.draw_frame(
                        D_curr=d_curr, best_D=d_best,
                        progress_value_curr=p_metric, progress_label_curr=progress_label,
                        progress_value_best=None, progress_label_best="Overall Best",
                        curr_cost=c_curr, best_cost=c_best
                    )
                else: # Single-panel
                    visualizer.draw_frame(
                        D_curr=d_curr, best_D=d_best,
                        progress_value_curr=None, progress_label_curr="",
                        progress_value_best=p_metric, progress_label_best=progress_label,
                        curr_cost=c_curr, best_cost=c_best
                    )
                canvas = visualizer.fig.canvas
                canvas.draw()
                width, height = canvas.get_width_height()
                buffer = canvas.buffer_rgba()
                image_array = np.frombuffer(buffer, dtype=np.uint8).reshape((height, width, 4))
                collected_frames.append(image_array.copy())
        active_callback = generic_emit_frame
    else:
        active_callback = None

    print(f"Starting {algorithm_name} for MDS ({num_vertices} vertices)...")
    if search_kwargs is None: search_kwargs = {}
    best_solution_list, best_solution_cost = algorithm_instance.search(
        callback=active_callback, **search_kwargs
    )
    print(f"{algorithm_name} search finished. Best solution size: {len(best_solution_list)}")

    if generate_gif:
        print(f"Collected {len(collected_frames)} frames for {algorithm_name} GIF.")
        algo_name_slug = algorithm_name.lower().replace(" ", "_").replace(".", "")
        
        param_string_for_filename = ""
        if current_config_params_for_gif_name: # Use passed params for uniqueness
            # Create a concise string from key parameters for the filename
            # Example: T100_a0.98_fsgreedy (fs for init_strategy)
            parts = []
            if 'initial_temp' in current_config_params_for_gif_name: parts.append(f"T{current_config_params_for_gif_name['initial_temp']:.0f}")
            if 'alpha' in current_config_params_for_gif_name: parts.append(f"a{current_config_params_for_gif_name['alpha']:.2f}")
            if 'pop_size' in current_config_params_for_gif_name: parts.append(f"P{current_config_params_for_gif_name['pop_size']}")
            if 'num_generations' in current_config_params_for_gif_name: parts.append(f"G{current_config_params_for_gif_name['num_generations']}")
            if 'init_strategy' in current_config_params_for_gif_name: parts.append(f"is{current_config_params_for_gif_name['init_strategy'][:3]}") # First 3 chars
            param_string_for_filename = "_" + "_".join(parts) if parts else ""

        base_filename = gif_base_filename_template.format(ALGO_NAME_SLUG=algo_name_slug)
        final_gif_name = base_filename + param_string_for_filename + ".gif"

        if gif_output_folder:
            os.makedirs(gif_output_folder, exist_ok=True)
            actual_gif_filepath = os.path.join(gif_output_folder, final_gif_name)
        else:
            actual_gif_filepath = final_gif_name

        if collected_frames:
            pil_images = [Image.fromarray(frame_arr) for frame_arr in collected_frames]
            duration_ms = 100
            if len(pil_images) < 20 and len(pil_images) > 0: duration_ms = 200
            try:
                pil_images[0].save(
                    actual_gif_filepath, save_all=True, append_images=pil_images[1:],
                    duration=duration_ms, loop=0, optimize=False
                )
                print(f"{algorithm_name} GIF saved as {actual_gif_filepath}")
                try:
                    with open(actual_gif_filepath, "rb") as f: data_gif = base64.b64encode(f.read()).decode("utf-8")
                    display(HTML(f'<h4>{algorithm_name} for MDS ({num_vertices}v) {param_string_for_filename.replace("_", " ")}</h4>'
                                 f'<img src="data:image/gif;base64,{data_gif}" alt="{algorithm_name} MDS Animation" />'))
                except Exception as display_err: print(f"Could not display {actual_gif_filepath}. Error: {display_err}")
            except Exception as e: print(f"Error saving/displaying {algorithm_name} GIF: {e}")
        else: print(f"No frames collected for {algorithm_name}, GIF not saved.")
        if visualizer and visualizer.fig: plt.close(visualizer.fig)

    return best_solution_list, best_solution_cost

In [7]:
def run_parameter_sweep(
    algorithm_class: type, # e.g., SimulatedAnnealingMDS or GeneticMDS
    algorithm_name: str,   # e.g., "Simulated Annealing"
    param_grid: Dict[str, List[Any]],
    fixed_constructor_params: Dict[str, Any], # Params always passed to constructor (e.g., adjacency_matrix)
    fixed_search_kwargs: Dict[str, Any],      # Fixed params for the search method
    num_runs_per_config: int = 1, # How many times to run each config (for stats)
    base_seed: int = 42,
    generate_gifs_for_batch: bool = False,
    gif_output_folder_batch: Optional[str] = None, # Folder for all GIFs from this sweep
    # Add a way to specify if a particular config should generate a GIF
    gif_configs_indices: Optional[List[int]] = None # List of indices of configs to generate GIF for
) -> pd.DataFrame:
    """
    Runs a parameter sweep for a given algorithm.
    """
    keys, values = zip(*param_grid.items())
    experiment_configs_raw = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Filter configurations if needed (e.g., for GA specific logic like elitism/elite_size)
    experiment_configs = []
    if algorithm_name == "Genetic Algorithm":
        for config in experiment_configs_raw:
            valid_config = True
            if not config.get('elitism', True) and config.get('elite_size', 0) > 0:
                config['elite_size'] = 0 # Standardize
            elif config.get('elitism', True) and config.get('elite_size', 0) >= config.get('pop_size', 1):
                valid_config = False
            elif config.get('elitism', True) and config.get('elite_size', 0) == 0 and 'elite_size' in config: # elite_size must be > 0 if elitism is True
                 valid_config = False

            if valid_config:
                # Avoid adding exact duplicate configurations if filtering logic creates them
                is_duplicate = False
                for existing_config in experiment_configs:
                    if config == existing_config:
                        is_duplicate = True
                        break
                if not is_duplicate:
                    experiment_configs.append(config)
    else: # For SA or other algorithms without such specific inter-parameter dependencies
        experiment_configs = experiment_configs_raw

    print(f"Total {algorithm_name} configurations to evaluate: {len(experiment_configs)}")
    if num_runs_per_config > 1:
        print(f"Each configuration will be run {num_runs_per_config} times with different seeds.")

    results_list = []

    for i, config_params in enumerate(experiment_configs):
        print(f"\n--- Evaluating {algorithm_name} Configuration {i+1}/{len(experiment_configs)} ---")
        print(f"Parameters: {config_params}")

        # For aggregated results over multiple runs
        current_config_costs = []
        current_config_sizes = []
        current_config_runtimes = []
        best_sol_for_this_config = None # Store the best solution from all runs of this config
        min_cost_for_this_config = float('inf')


        for run_idx in range(num_runs_per_config):
            # Determine if GIF should be generated for THIS specific run
            # Generates GIF if batch flag is true OR if this config index is in gif_configs_indices
            should_generate_this_gif = generate_gifs_for_batch or \
                                       (gif_configs_indices is not None and i in gif_configs_indices)

            current_run_seed = base_seed + i * num_runs_per_config + run_idx # Ensures unique seed per run

            print(f"  Run {run_idx + 1}/{num_runs_per_config} with seed {current_run_seed}...")

            # Prepare constructor and search arguments
            constructor_args = fixed_constructor_params.copy()
            search_args = fixed_search_kwargs.copy()

            # Override with parameters from the current grid configuration
            # Separate params for constructor vs search method based on typical usage
            temp_constructor_args = {}
            temp_search_args = {}

            for param_key, param_value in config_params.items():
                # Heuristic: if param is in SA/GA __init__ or common, put in constructor
                # Otherwise, assume it's for search. This is a bit fragile.
                # A better way is to explicitly list which params go where.
                if param_key in ['initial_temp', 'final_temp', 'alpha', 'iterations', # SA
                                 'pop_size', 'num_generations', 'tournament_size', # GA
                                 'crossover_rate', 'mutation_rate', 'elitism', 'elite_size', # GA
                                 'use_local_search']: # GA
                    temp_constructor_args[param_key] = param_value
                else: # e.g., 'init_strategy' for SA
                    temp_search_args[param_key] = param_value
            
            constructor_args.update(temp_constructor_args)
            constructor_args['seed'] = current_run_seed # Always set the seed for the run
            search_args.update(temp_search_args)
            
            # Instantiate algorithm
            algo_instance = algorithm_class(**constructor_args)

            start_time = time.time()
            try:
                best_sol, best_cost = run_and_visualize_algorithm(
                    algorithm_instance=algo_instance,
                    algorithm_name=algorithm_name,
                    adjacency_matrix_data=constructor_args['adjacency_matrix'], # Pass explicitly
                    num_vertices=constructor_args['adjacency_matrix'].shape[0], # Pass explicitly
                    search_kwargs=search_args,
                    generate_gif=should_generate_this_gif,
                    gif_output_folder=gif_output_folder_batch,
                    current_config_params_for_gif_name=config_params if should_generate_this_gif else None,
                    mode='local' # Or pass as a parameter to run_parameter_sweep
                )
                run_time = time.time() - start_time

                current_config_costs.append(best_cost)
                current_config_sizes.append(len(best_sol))
                current_config_runtimes.append(run_time)

                if best_cost < min_cost_for_this_config:
                    min_cost_for_this_config = best_cost
                    best_sol_for_this_config = best_sol # Store the actual best solution list

                print(f"    Finished in {run_time:.2f}s. Cost: {best_cost:.2f}, Size: {len(best_sol)}")

            except Exception as e:
                run_time = time.time() - start_time
                print(f"    ERROR during run {run_idx+1} for {algorithm_name} config {config_params} with seed {current_run_seed}: {e}")
                current_config_costs.append(np.nan)
                current_config_sizes.append(np.nan)
                current_config_runtimes.append(np.nan)

        # Aggregate results for this configuration
        result_entry = config_params.copy()
        result_entry['algorithm'] = algorithm_name
        result_entry['avg_cost'] = np.nanmean(current_config_costs) if current_config_costs else np.nan
        result_entry['std_cost'] = np.nanstd(current_config_costs) if current_config_costs else np.nan
        result_entry['min_cost_of_runs'] = np.nanmin(current_config_costs) if current_config_costs else np.nan
        result_entry['avg_size'] = np.nanmean(current_config_sizes) if current_config_sizes else np.nan
        result_entry['avg_runtime'] = np.nanmean(current_config_runtimes) if current_config_runtimes else np.nan
        result_entry['num_successful_runs'] = num_runs_per_config - np.sum(np.isnan(current_config_costs))
        result_entry['best_solution_example'] = best_sol_for_this_config # Store one example of the best solution found
        results_list.append(result_entry)

    results_df = pd.DataFrame(results_list)
    return results_df

# Main

##### In this part of the file, the adjacency matrix will first be loaded. Then, both the Simulated Annealing and Genetic Algorithm approaches will be executed using various parameter combinations and analyzed. The analysis will be enriched through an analysis of the GIF visualizations.

In [8]:
# Load the adjacency matrix from a file
adjacency_matrix = np.loadtxt('graph_1.txt', dtype=int)
print(f" Adjacency matrix shape: {adjacency_matrix.shape}")     

# Validate the adjacency matrix
if adjacency_matrix.ndim != 2 or adjacency_matrix.shape[0] != adjacency_matrix.shape[1]:
    raise ValueError("adjacency_matrix must be a square 2‑D array")    

N_VERTICES = adjacency_matrix.shape[0]

# Pandas version
# adjacency_matrix = pd.read_csv('graph_1.txt', sep=r'\s+', header=None)
# adjacency_matrix

# Define shared fixed parameters (common to all algorithms)
common_constructor_params = {
    'adjacency_matrix': adjacency_matrix,
    'penalty': N_VERTICES * 2.0
}

sa_fixed_search_kwargs = {}
ga_fixed_search_kwargs = {}

 Adjacency matrix shape: (112, 112)


## Simulated Annealing

In [10]:
# --- Run SA Parameter Sweep ---
sa_param_grid = {
    'initial_temp': [100.0, 200.0],
    'final_temp': [0.01],
    'alpha': [0.7, 0.85, 0.99],
    'iterations': [500],
    'init_strategy': ['greedy', 'random']
}

print("--- Starting SA Parameter Sweep ---")
sa_results_df = run_parameter_sweep(
    algorithm_class=SimulatedAnnealingMDS,
    algorithm_name="Simulated Annealing",
    param_grid=sa_param_grid,
    fixed_constructor_params=common_constructor_params,
    fixed_search_kwargs=sa_fixed_search_kwargs,
    num_runs_per_config=1,
    base_seed=42,
    generate_gifs_for_batch=False,
    gif_output_folder_batch="gifs/sa_runs",
    gif_configs_indices=[]
)

sa_results_df.to_csv("sa_sweep_results.csv", index=False)

--- Starting SA Parameter Sweep ---
Total Simulated Annealing configurations to evaluate: 12

--- Evaluating Simulated Annealing Configuration 1/12 ---
Parameters: {'initial_temp': 100.0, 'final_temp': 0.01, 'alpha': 0.7, 'iterations': 500, 'init_strategy': 'greedy'}
  Run 1/1 with seed 42...
Starting Simulated Annealing for MDS (112 vertices)...
Simulated Annealing search finished. Best solution size: 18
    Finished in 0.02s. Cost: 18.00, Size: 18

--- Evaluating Simulated Annealing Configuration 2/12 ---
Parameters: {'initial_temp': 100.0, 'final_temp': 0.01, 'alpha': 0.7, 'iterations': 500, 'init_strategy': 'random'}
  Run 1/1 with seed 43...
Starting Simulated Annealing for MDS (112 vertices)...
Simulated Annealing search finished. Best solution size: 81
    Finished in 0.01s. Cost: 81.00, Size: 81

--- Evaluating Simulated Annealing Configuration 3/12 ---
Parameters: {'initial_temp': 100.0, 'final_temp': 0.01, 'alpha': 0.85, 'iterations': 500, 'init_strategy': 'greedy'}
  Run 1/1

In [18]:
print("\n--- SA Parameter Sweep Results ---")
sa_results_df.rename(columns={'min_cost_of_runs': 'best_cost_found'}, inplace=True)

sa_results_filter_df = sa_results_df[
    ["initial_temp", "final_temp", "alpha", "iterations", "init_strategy", "best_cost_found"]
    ].sort_values(by="best_cost_found").reset_index(drop=True)

sa_results_filter_df.to_csv("sa_sweep_results_filter.csv", index=False)
sa_results_filter_df


--- SA Parameter Sweep Results ---


,initial_temp,final_temp,alpha,iterations,init_strategy,best_cost_found
0,100.0,0.01,0.70,500,greedy,18.0
1,100.0,0.01,0.85,500,greedy,18.0
2,200.0,0.01,0.70,500,greedy,18.0
3,100.0,0.01,0.99,500,greedy,18.0
4,200.0,0.01,0.99,500,greedy,18.0
5,200.0,0.01,0.85,500,greedy,18.0
6,200.0,0.01,0.99,500,random,23.0
7,100.0,0.01,0.99,500,random,25.0
8,200.0,0.01,0.70,500,random,67.0
9,200.0,0.01,0.85,500,random,67.0


#### GIF for SA

In [2]:
# # --- Grid A: Focus on initial_temp ---
# print("--- Starting SA Parameter Sweep: Grid A (Initial Temperature Demo) ---")
# sa_gif_param_grid_A = {
#     'initial_temp': [1.0, 100.0, 1000.0],
#     'final_temp': [0.01],
#     'alpha': [0.98],
#     'iterations': [1000],
#     'init_strategy': ['random']
# }

# sa_results_df_A = run_parameter_sweep(
#     algorithm_class=SimulatedAnnealingMDS,
#     algorithm_name="Simulated Annealing",
#     param_grid=sa_gif_param_grid_A,
#     fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=sa_fixed_search_kwargs,
#     num_runs_per_config=1, 
#     base_seed=100,        
#     generate_gifs_for_batch=True, 
#     gif_output_folder_batch="gifs/sa_didactic_initial_temp",
# )
# sa_results_df_A.to_csv("sa_sweep_results_grid_A.csv", index=False)


# # --- Grid B: Focus on final_temp ---
# print("\n\n--- Starting SA Parameter Sweep: Grid B (Final Temperature Demo) ---")
# sa_gif_param_grid_B = {
#     'initial_temp': [100.0],
#     'final_temp': [10.0, 0.1, 0.001],
#     'alpha': [0.98],
#     'iterations': [1000],
#     'init_strategy': ['greedy']
# }

# sa_results_df_B = run_parameter_sweep(
#     algorithm_class=SimulatedAnnealingMDS,
#     algorithm_name="SA_FinalTempDemo",
#     param_grid=sa_gif_param_grid_B,
#     fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=sa_fixed_search_kwargs,
#     num_runs_per_config=1,
#     base_seed=200,
#     generate_gifs_for_batch=True,
#     gif_output_folder_batch="gifs/sa_didactic_final_temp",
# )
# sa_results_df_B.to_csv("sa_sweep_results_grid_B.csv", index=False)


# # --- Grid C: Focus on alpha (Cooling Rate)
# print("\n\n--- Starting SA Parameter Sweep: Grid C (Cooling Rate Demo) ---")
# sa_gif_param_grid_C = {
#     'initial_temp': [100.0],
#     'final_temp': [0.01],
#     'alpha': [0.85, 0.95, 0.995],    
#     'iterations': [1000],          
#     'init_strategy': ['random']
# }

# sa_results_df_C = run_parameter_sweep(
#     algorithm_class=SimulatedAnnealingMDS,
#     algorithm_name="SA_CoolingRateDemo",
#     param_grid=sa_gif_param_grid_C,
#     fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=sa_fixed_search_kwargs,
#     num_runs_per_config=1,
#     base_seed=300, 
#     generate_gifs_for_batch=True,
#     gif_output_folder_batch="gifs/sa_didactic_cooling_rate",
# )
# sa_results_df_C.to_csv("sa_sweep_results_grid_C.csv", index=False)

## Genetic Algorithm

In [8]:
# --- Run GA Parameter Sweep ---
ga_param_grid = {
    'pop_size': [20, 50, 100],
    'num_generations': [20, 50],
    'tournament_size': [3, 10],
    'crossover_rate': [0.7, 0.9],
    'mutation_rate': [0.02, 0.05],
    'elitism': [True, False],
    'elite_size': [2, 10], 
    'use_local_search': [False, True]
}
ga_fixed_search_kwargs = {} # GA search method doesn't have extra specific kwargs in our example

print("\n\n--- Starting GA Parameter Sweep ---")
ga_results_df = run_parameter_sweep(
    algorithm_class=GeneticMDS,
    algorithm_name="Genetic Algorithm",
    param_grid=ga_param_grid,
    fixed_constructor_params=common_constructor_params,
    fixed_search_kwargs=ga_fixed_search_kwargs,
    num_runs_per_config=1,
    base_seed=2000,
    generate_gifs_for_batch=False,
    gif_output_folder_batch="gifs/ga_runs",
    gif_configs_indices=[] # Example: No specific GIFs for GA batch
)

ga_results_df.to_csv("ga_sweep_results.csv", index=False)



--- Starting GA Parameter Sweep ---
Total Genetic Algorithm configurations to evaluate: 288

--- Evaluating Genetic Algorithm Configuration 1/288 ---
Parameters: {'pop_size': 20, 'num_generations': 20, 'tournament_size': 3, 'crossover_rate': 0.7, 'mutation_rate': 0.02, 'elitism': True, 'elite_size': 2, 'use_local_search': False}
  Run 1/1 with seed 2000...
Starting Genetic Algorithm for MDS (112 vertices)...
Genetic Algorithm search finished. Best solution size: 41
    Finished in 0.04s. Cost: 41.00, Size: 41

--- Evaluating Genetic Algorithm Configuration 2/288 ---
Parameters: {'pop_size': 20, 'num_generations': 20, 'tournament_size': 3, 'crossover_rate': 0.7, 'mutation_rate': 0.02, 'elitism': True, 'elite_size': 2, 'use_local_search': True}
  Run 1/1 with seed 2001...
Starting Genetic Algorithm for MDS (112 vertices)...
Genetic Algorithm search finished. Best solution size: 18
    Finished in 2.41s. Cost: 18.00, Size: 18

--- Evaluating Genetic Algorithm Configuration 3/288 ---
Par

In [11]:
print("\n--- GA Parameter Sweep Results ---")
ga_results_df.rename(columns={'min_cost_of_runs': 'best_cost_found'}, inplace=True)
ga_results_df.sort_values(by="best_cost_found")

ga_results_filter_df = ga_results_df[
    ["pop_size", "num_generations", "tournament_size", "crossover_rate", "mutation_rate", "elitism", "elite_size", "use_local_search", "best_cost_found"]
    ].sort_values(by="best_cost_found").reset_index(drop=True)
   

ga_results_filter_df.to_csv("ga_sweep_results_filter.csv", index=False)
ga_results_filter_df


--- GA Parameter Sweep Results ---


,pop_size,num_generations,tournament_size,crossover_rate,mutation_rate,elitism,elite_size,use_local_search,best_cost_found
0,20,20,3,0.9,0.02,False,0,True,18.0
1,20,20,3,0.9,0.05,True,2,True,18.0
2,20,20,3,0.9,0.05,True,10,True,18.0
3,20,20,3,0.9,0.05,False,0,True,18.0
4,20,20,10,0.7,0.02,True,2,True,18.0
...,...,...,...,...,...,...,...,...,...
283,20,20,3,0.7,0.05,True,2,False,48.0
284,100,20,3,0.7,0.05,False,0,False,49.0
285,20,20,3,0.7,0.05,True,10,False,51.0
286,20,20,3,0.9,0.05,False,0,False,54.0


In [ ]:
# Select top 10 and bottom 5 (for PDF report)
top_10_df = ga_results_filter_df.head(10)
bottom_5_df = ga_results_filter_df.tail(5)

top_10_df.to_csv("csv_files/ga_top_10_results.csv", index=False)
bottom_5_df.to_csv("csv_files/ga_bottom_5_results.csv", index=False)

### GIF for GA

In [1]:
# # Initialize a list to hold all DataFrames
# all_ga_didactic_results_dfs = []

# # --- Grid GA-A: Focus on Population Size (`pop_size`) ---
# print("\n\n--- Running GA Parameter Sweep: Grid GA-A (Population Size Demo) ---")
# ga_gif_param_grid_A = {
#     'pop_size': [10, 50, 100],
#     'num_generations': [200],
#     'tournament_size': [3],
#     'crossover_rate': [0.9],
#     'mutation_rate': [0.02],
#     'elitism': [True],
#     'elite_size': [1],
#     'use_local_search': [False]
# }
# ga_results_df_A = run_parameter_sweep(
#     algorithm_class=GeneticMDS, algorithm_name="GA_PopSizeDemo",
#     param_grid=ga_gif_param_grid_A, fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=ga_fixed_search_kwargs, num_runs_per_config=1, base_seed=1000,
#     generate_gifs_for_batch=True, gif_output_folder_batch="gifs/ga_didactic_pop_size",
# )
# print("\n--- GA Sweep Results (Grid A: Population Size) ---")
# if not ga_results_df_A.empty and 'avg_cost' in ga_results_df_A.columns:
#     print(ga_results_df_A[['pop_size', 'avg_cost', 'min_cost_of_runs', 'avg_size']].to_string())
#     all_ga_didactic_results_dfs.append(ga_results_df_A.assign(grid_name='PopSizeDemo')) # Add to list
# else:
#     print(ga_results_df_A.to_string())
#     print("Grid A results DataFrame is empty or lacks expected columns.")


# # --- Grid GA-B: Focus on Mutation Rate (`mutation_rate`) ---
# print("\n\n--- Running GA Parameter Sweep: Grid GA-B (Mutation Rate Demo) ---")
# ga_gif_param_grid_B = {
#     'pop_size': [50], 'num_generations': [200], 'tournament_size': [3],
#     'crossover_rate': [0.9], 'mutation_rate': [0.001, 0.02, 0.1, 0.3],
#     'elitism': [True], 'elite_size': [2], 'use_local_search': [False]
# }
# ga_results_df_B = run_parameter_sweep(
#     algorithm_class=GeneticMDS, algorithm_name="GA_MutationRateDemo",
#     param_grid=ga_gif_param_grid_B, fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=ga_fixed_search_kwargs, num_runs_per_config=1, base_seed=2000,
#     generate_gifs_for_batch=True, gif_output_folder_batch="gifs/ga_didactic_mutation_rate",
# )
# print("\n--- GA Sweep Results (Grid B: Mutation Rate) ---")
# if not ga_results_df_B.empty and 'avg_cost' in ga_results_df_B.columns:
#     print(ga_results_df_B[['mutation_rate', 'avg_cost', 'min_cost_of_runs', 'avg_size']].to_string())
#     all_ga_didactic_results_dfs.append(ga_results_df_B.assign(grid_name='MutationRateDemo')) # Add to list
# else:
#     print(ga_results_df_B.to_string())
#     print("Grid B results DataFrame is empty or lacks expected columns.")


# # --- Grid GA-C: Focus on Elitism (`elitism` and `elite_size`) ---
# print("\n\n--- Running GA Parameter Sweep: Grid GA-C (Elitism Demo) ---")
# ga_gif_param_grid_C = {
#     'pop_size': [50], 'num_generations': [200], 'tournament_size': [3],
#     'crossover_rate': [0.9], 'mutation_rate': [0.02],
#     'elitism': [False, True], 'elite_size': [1, 5],
#     'use_local_search': [False]
# }
# ga_results_df_C = run_parameter_sweep(
#     algorithm_class=GeneticMDS, algorithm_name="GA_ElitismDemo",
#     param_grid=ga_gif_param_grid_C, fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=ga_fixed_search_kwargs, num_runs_per_config=1, base_seed=3000,
#     generate_gifs_for_batch=True, gif_output_folder_batch="gifs/ga_didactic_elitism",
# )
# print("\n--- GA Sweep Results (Grid C: Elitism) ---")
# if not ga_results_df_C.empty and 'avg_cost' in ga_results_df_C.columns:
#     print(ga_results_df_C[['elitism', 'elite_size', 'avg_cost', 'min_cost_of_runs', 'avg_size']].to_string())
#     all_ga_didactic_results_dfs.append(ga_results_df_C.assign(grid_name='ElitismDemo')) # Add to list
# else:
#     print(ga_results_df_C.to_string())
#     print("Grid C results DataFrame is empty or lacks expected columns.")


# # --- Grid GA-D: Focus on Selection Pressure (`tournament_size`) ---
# print("\n\n--- Running GA Parameter Sweep: Grid GA-D (Tournament Size Demo) ---")
# ga_gif_param_grid_D = {
#     'pop_size': [50], 'num_generations': [200], 'tournament_size': [2, 10, 25],
#     'crossover_rate': [0.9], 'mutation_rate': [0.02], 'elitism': [True],
#     'elite_size': [2], 'use_local_search': [False]
# }
# ga_results_df_D = run_parameter_sweep(
#     algorithm_class=GeneticMDS, algorithm_name="GA_TournamentDemo",
#     param_grid=ga_gif_param_grid_D, fixed_constructor_params=common_constructor_params,
#     fixed_search_kwargs=ga_fixed_search_kwargs, num_runs_per_config=1, base_seed=4000,
#     generate_gifs_for_batch=True, gif_output_folder_batch="gifs/ga_didactic_tournament_size",
# )
# print("\n--- GA Sweep Results (Grid D: Tournament Size) ---")
# if not ga_results_df_D.empty and 'avg_cost' in ga_results_df_D.columns:
#     print(ga_results_df_D[['tournament_size', 'avg_cost', 'min_cost_of_runs', 'avg_size']].to_string())
#     all_ga_didactic_results_dfs.append(ga_results_df_D.assign(grid_name='TournamentDemo')) # Add to list
# else:
#     print(ga_results_df_D.to_string())
#     print("Grid D results DataFrame is empty or lacks expected columns.")

# print("\n\nAll didactic GA GIF generation experiments complete.")

# # --- Combine all GA didactic results into a single DataFrame ---
# if all_ga_didactic_results_dfs:
#     combined_ga_didactic_df = pd.concat(all_ga_didactic_results_dfs, ignore_index=True)
#     print("\n\n--- Combined GA Didactic Experiment Results ---")
#     print(combined_ga_didactic_df.to_string())
#     # Optionally save the combined DataFrame to a single CSV
#     combined_ga_didactic_df.to_csv("ga_didactic_all_grids_results.csv", index=False)
#     print("\nCombined GA didactic results saved to ga_didactic_all_grids_results.csv")
# else:
#     print("\nNo GA didactic results were collected to combine.")